In [1]:
import numpy as np
import xgboost as xgb
import cryptoaml.datareader as cdr
from sklearn.metrics import f1_score
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score
from sklearn.metrics import confusion_matrix
from collections import Counter

from skmultiflow.drift_detection import DDM
from imblearn.over_sampling import SMOTE 
from imblearn.under_sampling import NeighbourhoodCleaningRule

In [18]:
elliptic = cdr.get_data("noaa_weather", config_file="configuration/data/data_config_smote.yaml")
data = elliptic.train_test_split(train_size=0.7)["ALL"]


# test_data = data.test_X

print(data.train_X.shape)


# print(data.train_y[data.train_X["label"] == 1])
# print(data.test_X.shape)


display(data.train_y.iloc[0] == 1)
display(data.test_X)

# # print(test_data.shape)

# noaa = cdr.get_data("noaa_weather", config_file="configuration/data/ell_data_config_ncr.yaml")
# dataset = noaa.train_test_split(train_size=0.7)["ALL"]

# # train_X = dataset.train_X
# # train_y = dataset.train_y
# test_X = dataset.test_X
# display(test_X)
# test_y = dataset.test_y

# import xgboost as xgb

# clf = xgb.XGBClassifier(random_state=58, subsample=0.5)
# clf.fit(train_X, train_y)
# y_pred = clf.predict(test_X)

# EXP_RESULT_PATH         = "persistence/experiment_2.4/results"
# BENCHMARK_RESULTS       = "{}/{}".format(EXP_RESULT_PATH, "benchmark_model_results.pkl")
# BENCHMARK_TUNED_RESULTS = "{}/{}".format(EXP_RESULT_PATH, "tuned_benchmark_results.pkl")
# DEFAULT_RESULTS         = "{}/{}".format(EXP_RESULT_PATH, "defaults_models_results.pkl")
# TUNED_RESULTS           = "{}/{}".format(EXP_RESULT_PATH, "tuned_models_results.pkl")


# from sklearn.metrics import accuracy_score
# accuracy_score(test_y, y_pred)

(17928, 9)


False

,temperature,dew_point,sea_level_pressure,visibility,avg_wind_speed,max_sustained_wind_speed,max_temperature,min_temperature,ts
17928,20.9,14.4,1025.3,18.2,5.4,9.9,25.2,15.3,425
17929,25.5,23.2,1029.2,5.2,1.6,4.9,29.1,23.2,425
17930,20.7,18.9,1032.8,10.3,2.4,7.0,27.1,14.2,425
17931,22.3,18.7,1023.9,11.8,2.9,8.9,33.1,13.1,425
17932,31.0,28.0,1018.8,14.7,3.0,9.9,36.0,26.2,425
...,...,...,...,...,...,...,...,...,...
23362,32.6,21.9,1022.7,7.0,12.4,26.0,41.0,24.8,606
23363,36.8,25.4,1014.9,7.0,10.0,19.0,57.2,21.2,606
23364,41.9,29.9,1010.0,7.0,4.7,9.9,62.6,28.4,606
23365,42.4,29.7,1011.3,7.0,3.3,8.9,51.8,32.0,606


In [3]:
# from skmultiflow.data import ConceptDriftStream
# from skmultiflow.evaluation import EvaluatePrequential


# # Adaptive XGBoost classifier parameters
# n_estimators = 30       # Number of members in the ensemble
# learning_rate = 0.3     # Learning rate or eta
# max_depth = 6           # Max depth for each tree in the ensemble
# max_window_size = 1000  # Max window size
# min_window_size = 1     # set to activate the dynamic window strategy
# detect_drift = False    # Enable/disable drift detection

# AXGBp = AdaptiveXGBoostClassifier(update_strategy='push',
#                                   n_estimators=n_estimators,
#                                   learning_rate=learning_rate,
#                                   max_depth=max_depth,
#                                   max_window_size=max_window_size,
#                                   min_window_size=min_window_size,
#                                   detect_drift=detect_drift)

# AXGBr = AdaptiveXGBoostClassifier(update_strategy='replace',
#                                   n_estimators=n_estimators,
#                                   learning_rate=learning_rate,
#                                   max_depth=max_depth,
#                                   max_window_size=max_window_size,
#                                   min_window_size=min_window_size,
#                                   detect_drift=detect_drift)
# print(data.shape)

In [4]:
# # display(train_data)
# # display(test_data)

# model = xgb.XGBClassifier()
# model.fit(train_data.iloc[:,:-1], train_data["class"])

# y_pred = model.predict(test_data.iloc[:,:-1])
# ts_f1 = round(f1_score(test_data["class"], y_pred, average='binary'), 3)
# print(ts_f1)

# true_test = []
# f1_scores = []
# predictions_test = []
# timestep_range = np.arange(35, 50)
# for timestep in timestep_range:
#     test = test_data[test_data["ts"] == timestep]
#     test_X = test.iloc[:,:-1]
#     test_y = test["class"]
    
#     y_pred = model.predict(test_X)
#     ts_f1 = round(f1_score(test_y, y_pred, average='binary'), 3)
#     print("Timestep: {} | F1-Score: {}".format(timestep, ts_f1))
#     f1_scores.append(ts_f1)
#     true_test.append(test_y)
#     predictions_test.append(y_pred)
        
# f1_score_test = f1_score(np.concatenate(true_test, axis=0),   
#                          np.concatenate(predictions_test, axis=0), 
#                          average='binary')
# print("F1-Score on test set: {}".format(round(f1_score_test, 3)))     

# recall_score_test = recall_score(np.concatenate(true_test, axis=0),   
#                                  np.concatenate(predictions_test, axis=0), 
#                                  average='binary')
# print("Recall on test set: {}".format(round(recall_score_test, 3)))      

# precision_score_test = precision_score(np.concatenate(true_test, axis=0),   
#                                        np.concatenate(predictions_test, axis=0), 
#                                        average='binary')
# print("Precision on test set: {}".format(round(precision_score_test, 3)))    

# confusion_matrix_test = confusion_matrix(np.concatenate(true_test, axis=0), 
#                                          np.concatenate(predictions_test, axis=0))
# print("Confusion_matrix: {}".format(confusion_matrix_test))    

In [5]:
# # display(train_data)
# # display(test_data)

# # ncr_X, ncr_y = ncr.fit_resample(train_data.iloc[:,:-1], train_data["class"])
# # sm = SMOTE()
# # X, y = sm.fit_resample(ncr_X, ncr_y)

# # ncr = NeighbourhoodCleaningRule(n_neighbors=3, threshold_cleaning=0.5)
# # ncr_X, ncr_y = ncr.fit_resample(train_data.iloc[:,:-1], train_data["class"])

# model = xgb.XGBClassifier()
# model.fit(train_data.iloc[:,:-1], train_data["class"])

# # model.fit(train_data.iloc[:,:-1], train_data["class"])

# max_timestep = test_data["ts"].max()
# true_test = []
# predictions_test = []
# timestep_range = np.arange(35, max_timestep)

# ensembles = []

# for timestep in timestep_range:

#     train = test_data[test_data["ts"] == timestep]
#     train_X = train.iloc[:,:-1]
#     train_y = train["class"]
    
#     if timestep == 35:
#         y_pred = model.predict(train_X)
#         ts_f1 = round(f1_score(train_y, y_pred, average='binary'), 3)     
#         print("Timestep: {} | F1-Score: {}".format(timestep, ts_f1))
#         true_test.append(train_y)
#         predictions_test.append(y_pred)
     
#     #ncr_X, ncr_y = ncr.fit_resample(train_X, train_y)
#     booster = model.get_booster()
#     model.fit(train_X, train_y, xgb_model=booster)
       
#     test = test_data[test_data["ts"] == timestep + 1]
#     test_X = test.iloc[:,:-1]
#     test_y = test["class"]
    
#     y_pred = model.predict(test_X)
#     ts_f1 = round(f1_score(test_y, y_pred, average='binary'), 3)
#     print("Timestep: {} | F1-Score: {}".format(timestep + 1, ts_f1))
#     true_test.append(test_y)
#     predictions_test.append(y_pred)
        
#     tmp_model = xgb.XGBClassifier()
#     tmp_model.fit(train_X, train_y)
#     ensembles.append(tmp_model) 
    
#     print("--------------------")
#     for i in range(len(ensembles) - 1):
#         tmp = ensembles[i]
#         tmp_booster = tmp.get_booster()
#         tmp.fit(train_X, train_y, xgb_model=tmp_booster)
#         ensembles[i] = tmp 
#         y_pred = tmp.predict(test_X)
#         f1_tmp = round(f1_score(test_y, y_pred, average='binary'), 3)
#         print(f1_tmp)
#     print("--------------------")

    
# #     y_pred = tmp_model.predict(test_X)
# #     ts_f1_tmp = round(f1_score(test_y, y_pred, average='binary'), 3)
    
# #     if ts_f1_tmp > ts_f1:
# #         print("TMP_Timestep: {} | F1-Score: {}".format(timestep + 1, ts_f1_tmp))

    
    
# f1_score_test = f1_score(np.concatenate(true_test, axis=0),   
#                          np.concatenate(predictions_test, axis=0), 
#                          average='binary')
# print("F1-Score on test set: {}".format(round(f1_score_test, 3)))     

# recall_score_test = recall_score(np.concatenate(true_test, axis=0),   
#                                  np.concatenate(predictions_test, axis=0), 
#                                  average='binary')
# print("Recall on test set: {}".format(round(recall_score_test, 3)))      

# precision_score_test = precision_score(np.concatenate(true_test, axis=0),   
#                                        np.concatenate(predictions_test, axis=0), 
#                                        average='binary')
# print("Precision on test set: {}".format(round(precision_score_test, 3)))    

# confusion_matrix_test = confusion_matrix(np.concatenate(true_test, axis=0), 
#                                          np.concatenate(predictions_test, axis=0))
# print("Confusion_matrix: {}".format(confusion_matrix_test))    